In [30]:
import pandas as pd
import urllib
import requests
from bs4 import BeautifulSoup
import re
import csv
import random
import numpy as np
import time

In [31]:
url = 'http://ws.isiknowledge.com/cps/openurl/service?url_ver=Z39.88-2004&rft_id=info:doi/10.1109/TVCG.2021.3114787'
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
}
response = requests.get(url=url, headers=headers)

In [32]:
if response.history:
    print("Request was redirected")
    for resp in response.history:
        print(resp.status_code, resp.url)
    print("Final destination:")
    print(response.status_code, response.url)
else:
    print("Request was not redirected")

Request was redirected
302 http://ws.isiknowledge.com/cps/openurl/service?url_ver=Z39.88-2004&rft_id=info:doi/10.1109/TVCG.2021.3114787
302 http://gateway.webofknowledge.com/gateway/Gateway.cgi?GWVersion=2&SrcApp=PARTNER_APP&SrcAuth=LinksAMR&KeyUT=WOS:000733959000058&DestLinkType=FullRecord&DestApp=ALL_WOS&UsrCustomerID=8703b88d69db6b417a9c0dc510538f44
302 http://www.webofknowledge.com?DestApp=WOSNX&DestParams=%2Fwos%2Fwoscc%2Ffull-record%2FWOS%3A000733959000058%3F&&DestFail=http%3A%2F%2Fwww.webofknowledge.com
Final destination:
200 https://access.clarivate.com/login?app=wos&detectSession=true&referrer=TARGET%3Dhttps%253A%252F%252Fwww.webofscience.com%252Fwos%253FIsProductCode%253DYes%2526Init%253DYes%2526DestParams%253D%25252Fwos%25252Fwoscc%25252Ffull-record%25252FWOS%25253A000733959000058%25253F%2526DestApp%253DWOSNX%2526Func%253DFrame%2526DestFail%253Dhttp%25253A%25252F%25252Fwww.webofknowledge.com%2526SrcApp%253DCR%2526SID%253D7Ev6Y2vwi12I4kEUu8M%26DestFail%3Dhttp%253A%252F%252Fww

In [33]:
url = response.history[-1].url

In [34]:
url

'http://www.webofknowledge.com?DestApp=WOSNX&DestParams=%2Fwos%2Fwoscc%2Ffull-record%2FWOS%3A000733959000058%3F&&DestFail=http%3A%2F%2Fwww.webofknowledge.com'

In [35]:
re.findall(r'(?<=2FWOS%3A)(.*)(?=%3F)', url)[0]

'000733959000058'

In [36]:
doi_wos_dict_list = []

In [37]:
def get_wos_id_from_doi(doi):
    url = 'http://ws.isiknowledge.com/cps/openurl/service?url_ver=Z39.88-2004&rft_id=info:doi/' + doi
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    }
    response = requests.get(url=url, headers=headers)
    wos_url = response.history[-1].url
    wos_id_list = re.findall(r'(?<=2FWOS%3A)(.*)(?=%3F)', wos_url)
    if wos_id_list:
        wos_id = wos_id_list[0]
    else:
        wos_id = np.NaN
    doi_wos_dict = {
        'DOI': doi,
        'WOS ID': wos_id
    }
    doi_wos_dict_list.append(doi_wos_dict)

In [38]:
"""Open `good_dois` and convert it to a list
"""

good_dois = open("../../data/processed/papers_to_study.txt", 'r')
reader = csv.reader(good_dois)
allRows = [row for row in reader]
dois = [i[0] for i in allRows]

In [39]:
"""randomly select 100 dois
"""
random.seed(1234)
random_dois = random.sample(dois, 100)

In [40]:
random_dois[0]

'10.1109/tvcg.2021.3114761'

In [41]:
for doi in dois:
    get_wos_id_from_doi(doi)
    time.sleep(2+random.uniform(0, 2))
    print(f'{dois.index(doi) + 1} is done')

1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
11 is done


KeyboardInterrupt: 

In [ ]:
# initiate a dataframe 
doi_wos_df_initiate = pd.DataFrame(columns=['DOI', 'WOS ID'])

In [ ]:
def build_df_from_dict_list(df, dict_list):
    """build df from a list of dictionaries
    
    Arguments:
       df: an empty df you just initiated
       
       dict_list: a list of dictionaries containing data you want to form a df
    
    Returns:
      The updated df
    """
    for i in dict_list:
        df_1 = pd.DataFrame([i])
        df = df.append(df_1, ignore_index=True)
    return df

In [ ]:
doi_wos_df = build_df_from_dict_list(doi_wos_df_initiate, doi_wos_dict_list)

In [ ]:
doi_wos_df

In [ ]:
na_rows = doi_wos_df[doi_wos_df.isnull().any(axis=1)]
na_rows

In [ ]:
len(na_rows)